# Protein Ligand Complex MD Setup tutorial using BioExcel Building Blocks (biobb)
### Based on the official Gromacs tutorial: http://www.mdtutorials.com/gmx/complex/index.html
***
This tutorial aims to illustrate the process of **setting up a simulation system** containing a **protein in complex with a ligand**, step by step, using the **BioExcel Building Blocks library (biobb)**. 
***
**Biobb modules** used:

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_model](https://github.com/bioexcel/biobb_model): Tools to model macromolecular structures.
 - [biobb_chemistry](https://github.com/bioexcel/biobb_chemistry): Tools to manipulate chemical data.
 - [biobb_gromacs](https://github.com/bioexcel/biobb_gromacs): Tools to setup and run Molecular Dynamics simulations.
 - [biobb_analysis](https://github.com/bioexcel/biobb_analysis): Tools to analyse Molecular Dynamics trajectories.
 - [biobb_structure_utils](https://github.com/bioexcel/biobb_structure_utils):  Tools to modify or extract information from a PDB structure file.
 
### Auxiliary libraries used

* [jupyter](https://jupyter.org/): Free software, open standards, and web services for interactive computing across all programming languages.
* [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
* [plotly](https://plot.ly/python/offline/): Python interactive graphing library integrated in Jupyter notebooks.
* [simpletraj](https://github.com/arose/simpletraj): Lightweight coordinate-only trajectory reader based on code from GROMACS, MDAnalysis and VMD.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_protein-complex_md_setup.git
cd biobb_wf_protein-complex_md_setup
conda env create -f conda_env/environment.yml
conda activate biobb_wf_protein-complex_md_setup
jupyter-notebook biobb_wf_protein-complex_md_setup/notebooks/biobb_wf_protein-complex_md_setup.ipynb
```

***
### Pipeline steps:
 1. [Input Parameters](#input)
 2. [Fetching PDB Structure](#fetch)
 3. [Fix Protein Structure](#fix)
 4. [Create Protein System Topology](#top)
 5. [Create ligand system topology](#ligtop)
 6. [Preparing Ligand Restraints](#restraints)
 7. [Create new protein-ligand complex structure file](#complex)
 8. [Create new protein-ligand complex topology file](#complextop)
 9. [Create Solvent Box](#box)
 10. [Fill the Box with Water Molecules](#water)
 11. [Adding Ions](#ions)
 12. [Energetically Minimize the System](#min)
 13. [Equilibrate the System (NVT)](#nvt)
 14. [Equilibrate the System (NPT)](#npt)
 15. [Free Molecular Dynamics Simulation](#free)
 16. [Post-processing and Visualizing Resulting 3D Trajectory](#post)
 17. [Output Files](#output)
 18. [Questions & Comments](#questions)
 
***
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***


## Pre-processing
Optional. Load variables declared during a previous session.

In [ ]:
%%script true
with open('notebook_variables.pkl', 'rb') as f:
        loaded_variables = pickle.load(f)

globals().update(loaded_variables)

<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein-ligand complex structure (e.g. 3HTB, [https://doi.org/10.2210/pdb3HTB/pdb](https://doi.org/10.2210/pdb3HTB/pdb))
 - **ligandCode**: Small molecule 3-letter code for the ligand structure (e.g. JZ4, [https://www.rcsb.org/ligand/JZ4](https://www.rcsb.org/ligand/JZ4))
 - **mol_charge**: Charge of the small molecule, needed to add hydrogen atoms.

In [ ]:
import nglview
import ipywidgets
import os
import zipfile
import sys

downloaded_pdb = 'aminostep2012.pdb'
ligandCode = "STM"
mol_charge = 0
nprocs = int(os.environ.get('PBS_NCPUS', '12'))

<a id="fetch"></a>
***
## Fetching PDB structure
A complex PDB file is used as starting structure. <br>
Splitting the molecule in **three different files**: 
- **proteinFile**: Protein structure
- **ligandFile**: Ligand structure
- **complexFile**: Protein-ligand complex structure 

***
**Building Blocks** used:
 - [Pdb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.pdb) from **biobb_io.api.pdb**
 - [ExtractHeteroAtoms](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.extract_heteroatoms) from **biobb_structure_utils.utils.extract_heteroatoms**
 - [ExtractMolecule](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.extract_molecule) from **biobb_structure_utils.utils.extract_molecule**
 - [CatPDB](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.cat_pdb) from **biobb_structure_utils.utils.cat_pdb**
***

In [ ]:
# Extracting Protein, Ligand and Protein-Ligand Complex to three different files
# Import module
from biobb_structure_utils.utils.extract_heteroatoms import extract_heteroatoms
from biobb_structure_utils.utils.extract_molecule import extract_molecule
from biobb_structure_utils.utils.cat_pdb import cat_pdb

# Create properties dict and inputs/outputs
proteinFile = "prot_" + downloaded_pdb
ligandFile = ligandCode+'.pdb'
complexFile = proteinFile.removesuffix('.pdb') + '_' + ligandCode+'.pdb'

prop = {
     'heteroatoms' : [{"name": "STM"}]
}

extract_heteroatoms(input_structure_path=downloaded_pdb,
     output_heteroatom_path=ligandFile,
     properties=prop)

extract_molecule(input_structure_path=downloaded_pdb,
     output_molecule_path=proteinFile)

print(proteinFile, ligandFile, complexFile)

cat_pdb(input_structure1=proteinFile,
       input_structure2=ligandFile,
       output_structure_path=complexFile)

2025-04-19 22:01:03,525 [MainThread  ] [INFO ]  Executing biobb_structure_utils.utils.extract_heteroatoms Version: 4.2.0
2025-04-19 22:01:03,530 [MainThread  ] [INFO ]  Copy: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/aminostep2012.pdb to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/efa85975-c8e4-48ff-8dbf-8040208ab3d8
2025-04-19 22:01:03,860 [MainThread  ] [INFO ]  Writting pdb to: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/efa85975-c8e4-48ff-8dbf-8040208ab3d8/STM.pdb
2025-04-19 22:01:03,866 [MainThread  ] [INFO ]  Removed: ['/home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/efa85975-c8e4-48ff-8dbf-8040208ab3d8']
2025-04-19 22:01:03,873 [MainThread  ] [INFO ]  Executing biobb_structure_utils.utils.extract_molecule Version: 4.2.0
2025-04-19 22

0

### Visualizing 3D structures
Visualizing the generated **PDB structures** using **NGL**:  
- **Protein structure** (Left)
- **Ligand structure** (Center)
- **Protein-ligand complex** (Right)  

In [ ]:
%%script true
# Show structures: protein, ligand and protein-ligand complex
view1 = nglview.show_structure_file(proteinFile)
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(ligandFile)
view2.add_representation(repr_type='ball+stick')
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.camera='orthographic'
view2
view3 = nglview.show_structure_file(complexFile)
view3.add_representation(repr_type='licorice', radius='.5', selection=ligandCode)
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.camera='orthographic'
view3
ipywidgets.HBox([view1, view2, view3])

<a id="fix"></a>
***
## Fix protein structure
**Checking** and **fixing** (if needed) the protein structure:<br>
- **Modeling** **missing side-chain atoms**, modifying incorrect **amide assignments**, choosing **alternative locations**.<br>
- **Checking** for missing **backbone atoms**, **heteroatoms**, **modified residues** and possible **atomic clashes**.

***
**Building Blocks** used:
 - [FixSideChain](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_side_chain) from **biobb_model.model.fix_side_chain**
***

In [4]:
# Check & Fix Protein Structure
# Import module
from biobb_model.model.fix_side_chain import fix_side_chain

# Create prop dict and inputs/outputs
fixed_pdb = proteinFile.removesuffix('.pdb')+'_fixed.pdb'

# Create and launch bb
fix_side_chain(input_pdb_path=proteinFile,
             output_pdb_path=fixed_pdb)

2025-04-19 22:01:32,536 [MainThread  ] [INFO ]  Executing biobb_model.model.fix_side_chain Version: 4.2.0
2025-04-19 22:01:32,541 [MainThread  ] [INFO ]  Copy: prot_aminostep2012.pdb to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/4b93193b-fd53-4879-9e2b-cc47e54aa9e1
2025-04-19 22:01:32,544 [MainThread  ] [INFO ]  check_structure -i /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/4b93193b-fd53-4879-9e2b-cc47e54aa9e1/prot_aminostep2012.pdb -o /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/4b93193b-fd53-4879-9e2b-cc47e54aa9e1/prot_aminostep2012_fixed.pdb --force_save fixside --fix ALL

2025-04-19 22:01:33,040 [MainThread  ] [INFO ]  Exit code 0

2025-04-19 22:01:33,041 [MainThread  ] [INFO ]  ================================================================================
=                   BioBB structure ch

0

<a id="top"></a>
***
## Create protein system topology

**Extra Step** The protein needs to be deprotonated for this to work.


In [6]:
import sys

def deprotonate_pdb(input_file, output_file):
    """
    Remove hydrogen atoms from a PDB file.

    Parameters:
        input_file (str): Path to the input PDB file.
        output_file (str): Path where the deprotonated PDB file will be saved.

    The function reads the input PDB file line by line. For lines that start with "ATOM" 
    or "HETATM", it checks the element column (columns 77-78) to see if it is a hydrogen (H).
    In cases where the element field is missing or ambiguous, the atom name (columns 13-16)
    is examined. If a hydrogen is detected, the line is skipped; otherwise, the line is written 
    to the output file unchanged. Note that the PDB format is fixed-width, so this script assumes 
    that the file follows standard formatting.
    """
    with open(input_file, "r") as f:
        lines = f.readlines()

    with open(output_file, "w") as outf:
        for line in lines:
            # Only check lines that define atoms.
            if line.startswith("ATOM") or line.startswith("HETATM"):
                # Extract the element symbol (columns 77-78) if possible.
                element = line[76:78].strip() if len(line) >= 78 else ""
                # If the element symbol is missing, fallback to the atom name (columns 13-16).
                atom_name = line[12:16].strip() if len(line) >= 16 else ""
                # Determine if this atom is hydrogen:
                if element.upper() == "H" or (not element and atom_name and atom_name[0].upper() == "H"):
                    continue  # Skip writing hydrogen atoms.
            # Write all non-hydrogen lines (and non-ATOM/HETATM lines) to the output.
            outf.write(line)

# Deprotonate the fixed PDB file
deprotonated_pdb = fixed_pdb.removesuffix('.pdb')+'_deprotonated.pdb'
deprotonate_pdb(fixed_pdb, deprotonated_pdb)
# Show deprotonated structure
view4 = nglview.show_structure_file(deprotonated_pdb)
view4._remote_call('setSize', target='Widget', args=['350px','400px'])
view4.camera='orthographic'
view4

NGLWidget()

**Building GROMACS topology** corresponding to the protein structure.<br>
Force field used in this tutorial is [**amber99sb-ildn**](https://dx.doi.org/10.1002%2Fprot.22711): AMBER **parm99** force field with **corrections on backbone** (sb) and **side-chain torsion potentials** (ildn). Water molecules type used in this tutorial is [**spc/e**](https://pubs.acs.org/doi/abs/10.1021/j100308a038).<br>
Adding **hydrogen atoms** if missing. Automatically identifying **disulfide bridges**. <br>

Generating two output files: 
- **GROMACS structure** (gro file)
- **GROMACS topology** ZIP compressed file containing:
    - *GROMACS topology top file* (top file)
    - *GROMACS position restraint file/s* (itp file/s)
***
**Building Blocks** used:
 - [Pdb2gmx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.pdb2gmx) from **biobb_gromacs.gromacs.pdb2gmx**
***

In [7]:
# Create Protein system topology
# Import module
from biobb_gromacs.gromacs.pdb2gmx import pdb2gmx

# Create inputs/outputs
output_pdb2gmx_gro = proteinFile.removesuffix('.pdb')+'_pdb2gmx.gro'
output_pdb2gmx_top_zip = proteinFile.removesuffix('.pdb')+'_pdb2gmx_top.zip'
prop = {
    'force_field' : 'amber99sb-ildn',
    'water_type': 'spce'
}

# Create and launch bb
pdb2gmx(input_pdb_path=deprotonated_pdb,
        output_gro_path=output_pdb2gmx_gro,
        output_top_zip_path=output_pdb2gmx_top_zip,
        properties=prop, restart=True)

2025-04-19 22:05:19,367 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.pdb2gmx Version: 4.2.0
2025-04-19 22:05:19,371 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_fixed_deprotonated.pdb to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/0509961e-3424-4a36-a49e-c6d2275a5484
2025-04-19 22:05:19,375 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright pdb2gmx -f /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/0509961e-3424-4a36-a49e-c6d2275a5484/prot_aminostep2012_fixed_deprotonated.pdb -o /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/0509961e-3424-4a36-a49e-c6d2275a5484/prot_aminostep2012_pdb2gmx.gro -p p2g.top -water spce -ff amber99sb-ildn -i posre.itp

2025-04-19 22:05:19,878 [MainThread  ] [INFO ]  Exit code 0

2025-04-19 22:05:19,878 [MainThread  ] [INFO ]  Using the Amber99sb-ildn force field in

0

<a id="ligtop"></a>
***
## Create ligand system topology
**Building GROMACS topology** corresponding to the ligand structure.<br>
Force field used in this tutorial step is **amberGAFF**: [General AMBER Force Field](http://ambermd.org/antechamber/gaff.html), designed for rational drug design.<br>
- [Step 1](#ligandTopologyStep1): Add **hydrogen atoms** if missing.
- [Step 2](#ligandTopologyStep2): **Energetically minimize the system** with the new hydrogen atoms. 
- [Step 3](#ligandTopologyStep3): Generate **ligand topology** (parameters). 
***
**Building Blocks** used:
 - [ReduceAddHydrogens](https://biobb-chemistry.readthedocs.io/en/latest/ambertools.html#module-ambertools.reduce_add_hydrogens) from **biobb_chemistry.ambertools.reduce_add_hydrogens**
 - [BabelMinimize](https://biobb-chemistry.readthedocs.io/en/latest/babelm.html#module-babelm.babel_minimize) from **biobb_chemistry.babelm.babel_minimize** 
 - [AcpypeParamsGMX](https://biobb-chemistry.readthedocs.io/en/latest/acpype.html#module-acpype.acpype_params_gmx) from **biobb_chemistry.acpype.acpype_params_gmx** 
***

<a id="ligandTopologyStep1"></a>
### Step 1: Add **hydrogen atoms**

Note that this is not needed as ligand was already protonated during docking.

In [8]:
# Create Ligand system topology, STEP 1
# Reduce_add_hydrogens: add Hydrogen atoms to a small molecule (using Reduce tool from Ambertools package)
# Import module
from biobb_chemistry.ambertools.reduce_add_hydrogens import reduce_add_hydrogens

# Create prop dict and inputs/outputs
output_reduce_h = ligandCode+'.pdb' 

<a id="ligandTopologyStep2"></a>
### Step 2: **Energetically minimize the system** with the new hydrogen atoms. 

In [9]:
# Create Ligand system topology, STEP 2
# Babel_minimize: Structure energy minimization of a small molecule after being modified adding hydrogen atoms
# Import module
from biobb_chemistry.babelm.babel_minimize import babel_minimize

# Create prop dict and inputs/outputs
output_babel_min = ligandCode+'.H.min.mol2'                              
prop = {
    'method' : 'sd',
    'criteria' : '1e-10',
    'force_field' : 'GAFF'
}


# Create and launch bb
babel_minimize(input_path=output_reduce_h,
              output_path=output_babel_min,
              properties=prop)

2025-04-19 22:07:08,312 [MainThread  ] [INFO ]  Executing biobb_chemistry.babelm.babel_minimize Version: 4.2.0
2025-04-19 22:07:08,316 [MainThread  ] [INFO ]  Copy: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/STM.pdb to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/6bde5f29-1741-4ff0-9cfa-3376fdc4b3d6
2025-04-19 22:07:08,319 [MainThread  ] [INFO ]  Hydrogens  is not correct, assigned default value: False
2025-04-19 22:07:08,322 [MainThread  ] [INFO ]  Steps  is not correct, assigned default value: 2500
2025-04-19 22:07:08,324 [MainThread  ] [INFO ]  Cut-off  is not correct, assigned default value: False
2025-04-19 22:07:08,327 [MainThread  ] [INFO ]  Rvdw  is not correct, assigned default value: 6.0
2025-04-19 22:07:08,330 [MainThread  ] [INFO ]  Rele  is not correct, assigned default value: 10.0
2025-04-19 22:07:08,332 [MainThread  ] [INFO ]  Frequency  is not co

0

### Visualizing 3D structures
Visualizing the small molecule generated **PDB structures** using **NGL**:  
- **Original Ligand Structure** (Left)
- **Ligand Structure with hydrogen atoms added** (with Reduce program) (Center)
- **Ligand Structure with hydrogen atoms added** (with Reduce program), **energy minimized** (with Open Babel) (Right) 

In [ ]:
%%script true
# Show different structures generated (for comparison)

view1 = nglview.show_structure_file(ligandFile)
view1.add_representation(repr_type='ball+stick')
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(output_reduce_h)
view2.add_representation(repr_type='ball+stick')
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.camera='orthographic'
view2
view3 = nglview.show_structure_file(output_babel_min)
view3.add_representation(repr_type='ball+stick')
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.camera='orthographic'
view3
ipywidgets.HBox([view1, view2, view3])

<a id="ligandTopologyStep3"></a>
### Step 3: Generate **ligand topology** (parameters).

In [11]:
# Create Ligand system topology, STEP 3
# Acpype_params_gmx: Generation of topologies for GROMACS with ACPype
# Import module
from biobb_chemistry.acpype.acpype_params_gmx import acpype_params_gmx

# Create prop dict and inputs/outputs
output_acpype_gro = ligandCode+'params.gro'
output_acpype_itp = ligandCode+'params.itp'
output_acpype_top = ligandCode+'params.top'
output_acpype = ligandCode+'params'
prop = {
    'basename' : output_acpype,
    'charge' : mol_charge
}

# Create and launch bb
acpype_params_gmx(input_path=output_babel_min, 
                output_path_gro=output_acpype_gro,
                output_path_itp=output_acpype_itp,
                output_path_top=output_acpype_top,
                properties=prop)

2025-04-19 22:07:56,059 [MainThread  ] [INFO ]  Executing biobb_chemistry.acpype.acpype_params_gmx Version: 4.2.0
2025-04-19 22:07:56,063 [MainThread  ] [INFO ]  Copy: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/STM.H.min.mol2 to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/11dfcccc-0d4b-4330-8b47-86f03c566253
2025-04-19 22:07:56,066 [MainThread  ] [INFO ]  acpype -i /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/11dfcccc-0d4b-4330-8b47-86f03c566253/STM.H.min.mol2 -b STMparams.9LzCxE -n 0

2025-04-19 22:26:56,202 [MainThread  ] [INFO ]  Exit code 0

2025-04-19 22:26:56,203 [MainThread  ] [INFO ]  ==========================================================================
| ACPYPE: AnteChamber PYthon Parser interfacE v. 2022.6.6 (c) 2025 AWSdS |
==> ... charge set to 0
==> Executing Antechamber...
==> * An

0

<a id="restraints"></a>
***
## Preparing Ligand Restraints
In subsequent steps of the pipeline, such as the equilibration stages of the **protein-ligand complex** system, it is recommended to apply some **restraints** to the small molecule, to avoid a possible change in position due to protein repulsion. **Position restraints** will be applied to the ligand, using a **force constant of 1000 KJ/mol\*nm^2** on the three coordinates: x, y and z. In this steps the **restriction files** will be created and integrated in the **ligand topology**.
- [Step 1](#restraintsStep1): Creating an index file with a new group including just the **small molecule heavy atoms**.
- [Step 2](#restraintsStep2): Generating the **position restraints** file.
***
**Building Blocks** used:
 - [MakeNdx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.make_ndx) from **biobb_gromacs.gromacs.make_ndx** 
 - [Genrestr](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.genrestr) from **biobb_gromacs.gromacs.genrestr** 
***

<a id="restraintsStep1"></a>
### Step 1: Creating an index file for the small molecule heavy atoms

In [ ]:
# MakeNdx: Creating import sys

from biobb_gromacs.gromacs.make_ndx import make_ndx

# Create prop dict and inputs/outputs
output_ligand_ndx = ligandCode+'_index.ndx'
prop = {
    'selection': "0 & ! a H*"
}

# Create and launch bb
make_ndx(input_structure_path=output_acpype_gro,
        output_ndx_path=output_ligand_ndx,
        properties=prop)

2025-04-19 22:27:10,551 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.make_ndx Version: 4.2.0
2025-04-19 22:27:10,553 [MainThread  ] [INFO ]  Copy: STMparams.gro to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/a46d5184-a5a9-4fbb-9d2a-b129df952c75
2025-04-19 22:27:10,554 [MainThread  ] [INFO ]  Copy: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/d4a7bd93-81cc-4573-89aa-5f8ba74598c3.stdin to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/a46d5184-a5a9-4fbb-9d2a-b129df952c75
2025-04-19 22:27:10,554 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright make_ndx -f /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/a46d5184-a5a9-4fbb-9d2a-b129df952c75/STMparams.gro -o /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/

0

<a id="restraintsStep2"></a>
### Step 2: Generating the position restraints file

In [13]:
# Genrestr: Generating the position restraints file
from biobb_gromacs.gromacs.genrestr import genrestr

# Create prop dict and inputs/outputs
output_restraints_top = ligandCode+'_posres.itp'
prop = {
    'force_constants': "1000 1000 1000",
    'restrained_group': "System_&_!H*"
}

# Create and launch bb
genrestr(input_structure_path=output_acpype_gro,
         input_ndx_path=output_ligand_ndx,
         output_itp_path=output_restraints_top,
         properties=prop)

2025-04-19 22:27:20,947 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.genrestr Version: 4.2.0
2025-04-19 22:27:20,950 [MainThread  ] [INFO ]  Copy: STMparams.gro to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/c39c20b5-3829-471b-9f09-00df6a9150ee
2025-04-19 22:27:20,952 [MainThread  ] [INFO ]  Copy: STM_index.ndx to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/c39c20b5-3829-471b-9f09-00df6a9150ee
2025-04-19 22:27:20,954 [MainThread  ] [INFO ]  Copy: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/697debec-056c-462f-bdde-cfcb6576b1cf.stdin to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/c39c20b5-3829-471b-9f09-00df6a9150ee
2025-04-19 22:27:20,956 [MainThread  ] [INFO ]  gmx  -nobackup -nocopyright genrestr -f /home/daneel/gitrepos/groma

0

<a id="complex"></a>
***
## Create new protein-ligand complex structure file
Building new **protein-ligand complex** PDB file with:
- The new **protein system** with fixed problems from *Fix Protein Structure* step and hydrogens atoms added from *Create Protein System Topology* step.
- The new **ligand system** with hydrogens atoms added from *Create Ligand System Topology* step. 

This new structure is needed for **GROMACS** as it is **force field-compliant**, it **has all the new hydrogen atoms**, and the **atom names are matching the newly generated protein and ligand topologies**.
***
**Building Blocks** used:
 - [GMXTrjConvStr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_trjconv_str) from **biobb_analysis.gromacs.gmx_trjconv_str**
 - [CatPDB](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.cat_pdb) from **biobb_structure_utils.utils.cat_pdb**
***

In [14]:
# biobb analysis module
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str
from biobb_structure_utils.utils.cat_pdb import cat_pdb

# Convert gro (with hydrogens) to pdb (PROTEIN)
proteinFile_H = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_complex_H.pdb'
prop = {
    'selection' : 'System'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_pdb2gmx_gro,
              input_top_path=output_pdb2gmx_gro,
              output_str_path=proteinFile_H, 
              properties=prop)

# Convert gro (with hydrogens) to pdb (LIGAND)
ligandFile_H = ligandCode+'_complex_H.pdb'
prop = {
    'selection' : 'System'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_acpype_gro,
              input_top_path=output_acpype_gro,
              output_str_path=ligandFile_H, 
              properties=prop)


# Concatenating both PDB files: Protein + Ligand
complexFile_H = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_H.pdb'

# Create and launch bb
cat_pdb(input_structure1=proteinFile_H,
       input_structure2=ligandFile_H,
       output_structure_path=complexFile_H)

2025-04-19 22:28:57,413 [MainThread  ] [INFO ]  Executing biobb_analysis.gromacs.gmx_trjconv_str Version: 4.2.0
2025-04-19 22:28:57,420 [MainThread  ] [INFO ]  Copy: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/prot_aminostep2012_pdb2gmx.gro to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/edc21868-562e-429e-aa38-571a6863320f
2025-04-19 22:28:57,425 [MainThread  ] [INFO ]  Copy: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/prot_aminostep2012_pdb2gmx.gro to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/edc21868-562e-429e-aa38-571a6863320f
2025-04-19 22:28:57,429 [MainThread  ] [INFO ]  Copy: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/eb605e09-546f-4aea-b05b-1c17f7a7f697.stdin to /ho

0

<a id="complextop"></a>
***
## Create new protein-ligand complex topology file
Building new **protein-ligand complex** GROMACS topology file with:
- The new **protein system** topology generated from *Create Protein System Topology* step.
- The new **ligand system** topology generated from *Create Ligand System Topology* step. 

NOTE: From this point on, the **protein-ligand complex structure and topology** generated can be used in a regular MD setup.
***
**Building Blocks** used:
 - [AppendLigand](https://biobb-gromacs.readthedocs.io/en/latest/gromacs_extra.html#gromacs-extra-append-ligand-module) from **biobb_gromacs.gromacs_extra.append_ligand**
***

In [15]:
# AppendLigand: Append a ligand to a GROMACS topology
# Import module
from biobb_gromacs.gromacs_extra.append_ligand import append_ligand

# Create prop dict and inputs/outputs
output_complex_top = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_complex.top.zip'

posresifdef = "POSRES_"+ligandCode.upper()
prop = {
    'posres_name': posresifdef
}

# Create and launch bb
append_ligand(input_top_zip_path=output_pdb2gmx_top_zip,
             input_posres_itp_path=output_restraints_top,
             input_itp_path=output_acpype_itp, 
             output_top_zip_path=output_complex_top,
             properties=prop)

2025-04-19 22:29:18,829 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs_extra.append_ligand Version: 4.2.0
2025-04-19 22:29:18,851 [MainThread  ] [INFO ]  Extracting: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/prot_aminostep2012_pdb2gmx_top.zip
2025-04-19 22:29:18,853 [MainThread  ] [INFO ]  to:
2025-04-19 22:29:18,855 [MainThread  ] [INFO ]  ['873c963e-5d52-4271-a8e0-5b8e4529bd13/p2g.top', '873c963e-5d52-4271-a8e0-5b8e4529bd13/p2g_Protein_chain_A.itp', '873c963e-5d52-4271-a8e0-5b8e4529bd13/p2g_Protein_chain_B.itp', '873c963e-5d52-4271-a8e0-5b8e4529bd13/posre_Protein_chain_A.itp', '873c963e-5d52-4271-a8e0-5b8e4529bd13/posre_Protein_chain_B.itp']
2025-04-19 22:29:18,858 [MainThread  ] [INFO ]  Unzipping: 
2025-04-19 22:29:18,861 [MainThread  ] [INFO ]  prot_aminostep2012_pdb2gmx_top.zip
2025-04-19 22:29:18,864 [MainThread  ] [INFO ]  To: 
2025-04-19 22:29:18,867 [MainThread  ] [INFO ]  873c963e-5d52-4271-a8e0-5b8e45

0

<a id="box"></a>
***
## Create solvent box
Define the unit cell for the **protein-ligand complex** to fill it with water molecules.<br>
**Truncated octahedron** box is used for the unit cell. This box type is the one which best reflects the geometry of the solute/protein, in this case a **globular protein**, as it approximates a sphere. It is also convenient for the computational point of view, as it accumulates **less water molecules at the corners**, reducing the final number of water molecules in the system and making the simulation run faster.<br> A **protein to box** distance of **0.8 nm** is used, and the protein is **centered in the box**.  

***
**Building Blocks** used:
 - [Editconf](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.editconf) from **biobb_gromacs.gromacs.editconf** 
***

In [16]:
# Editconf: Create solvent box
# Import module
from biobb_gromacs.gromacs.editconf import editconf

# Create prop dict and inputs/outputs
output_editconf_gro = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_complex_editconf.gro'

prop = {
    'box_type': 'octahedron',
    'distance_to_molecule': 0.8
}

# Create and launch bb
editconf(input_gro_path=complexFile_H, 
         output_gro_path=output_editconf_gro,
         properties=prop)

2025-04-19 22:29:31,405 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.editconf Version: 4.2.0
2025-04-19 22:29:31,410 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_H.pdb to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/98af90a1-5fc6-45b3-9af6-063f919c8af0
2025-04-19 22:29:31,413 [MainThread  ] [INFO ]  Distance of the box to molecule:   0.80
2025-04-19 22:29:31,417 [MainThread  ] [INFO ]  Centering molecule in the box.
2025-04-19 22:29:31,421 [MainThread  ] [INFO ]  Box type: octahedron
2025-04-19 22:29:31,423 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright editconf -f /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/98af90a1-5fc6-45b3-9af6-063f919c8af0/prot_aminostep2012_STM_H.pdb -o /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/98af90a1-5fc6-45b3-9af6-063f919c8af0/prot_aminostep2012_STM_

0

<a id="water"></a>
***
## Fill the box with water molecules
Fill the unit cell for the **protein-ligand complex** with water molecules.<br>
The solvent type used is the default **Simple Point Charge water (SPC)**, a generic equilibrated 3-point solvent model. 

***
**Building Blocks** used:
 - [Solvate](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.solvate) from **biobb_gromacs.gromacs.solvate** 
***

In [17]:
# Solvate: Fill the box with water molecules
from biobb_gromacs.gromacs.solvate import solvate

# Create prop dict and inputs/outputs
output_solvate_gro = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_solvate.gro'
output_solvate_top_zip = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_solvate_top.zip'

# Create and launch bb
solvate(input_solute_gro_path=output_editconf_gro,
        output_gro_path=output_solvate_gro,
        input_top_zip_path=output_complex_top,
        output_top_zip_path=output_solvate_top_zip)

2025-04-19 22:29:59,242 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.solvate Version: 4.2.0
2025-04-19 22:29:59,247 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_complex_editconf.gro to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/21ffc817-3b10-4336-b22e-24084a17cf61
2025-04-19 22:29:59,266 [MainThread  ] [INFO ]  Extracting: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/prot_aminostep2012_STM_complex.top.zip
2025-04-19 22:29:59,268 [MainThread  ] [INFO ]  to:
2025-04-19 22:29:59,270 [MainThread  ] [INFO ]  ['2f423064-d19f-4984-8f65-5fb340a3560f/STM_posres.itp', '2f423064-d19f-4984-8f65-5fb340a3560f/STMparams.itp', '2f423064-d19f-4984-8f65-5fb340a3560f/ligand.top', '2f423064-d19f-4984-8f65-5fb340a3560f/p2g_Protein_chain_A.itp', '2f423064-d19f-4984-8f65-5fb340a3560f/p2g_Protein_chain_B.itp', '2f423064-d19f-4984-8f65-5fb340a3560f/posre_Protein_chain

0

### Visualizing 3D structure
Visualizing the **protein-ligand complex** with the newly added **solvent box** using **NGL**<br>
Note the **octahedral box** filled with **water molecules** surrounding the **protein structure**, which is **centered** right in the middle of the box.

In [ ]:
%%script true
#Show protein
view = nglview.show_structure_file(output_solvate_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='licorice', radius='.5', selection=ligandCode)
view.add_representation(repr_type='line', linewidth='1', selection='SOL', opacity='.3')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<a id="ions"></a>
***
## Adding ions
Add ions to neutralize the **protein-ligand complex** and reach a desired ionic concentration.
- [Step 1](#ionsStep1): Creating portable binary run file for ion generation
- [Step 2](#ionsStep2): Adding ions to **neutralize** the system and reach a **0.05 molar ionic concentration**
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Genion](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.genion) from **biobb_gromacs.gromacs.genion** 
***

<a id="ionsStep1"></a>
### Step 1: Creating portable binary run file for ion generation

In [19]:
# Grompp: Creating portable binary run file for ion generation
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
prop = {
    'mdp':{
        'nsteps':'5000'
    },
    'simulation_type':'ions',
    'maxwarn': 1
}
output_gppion_tpr = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_complex_gppion.tpr'

# Create and launch bb
grompp(input_gro_path=output_solvate_gro,
       input_top_zip_path=output_solvate_top_zip, 
       output_tpr_path=output_gppion_tpr,
       properties=prop)

2025-04-19 22:30:44,727 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.grompp Version: 4.2.0
2025-04-19 22:30:44,736 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_solvate.gro to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/2f950185-b1d7-4819-8fd4-b6424193c437
2025-04-19 22:30:44,758 [MainThread  ] [INFO ]  Extracting: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/prot_aminostep2012_STM_solvate_top.zip
2025-04-19 22:30:44,760 [MainThread  ] [INFO ]  to:
2025-04-19 22:30:44,762 [MainThread  ] [INFO ]  ['/home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/2f950185-b1d7-4819-8fd4-b6424193c437/STM_posres.itp', '/home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/2f950185-b1d7-4819-8fd4-b6424193c437/STMparams.itp', '/home/daneel/gitrepos/gromacs_s

0

<a id="ionsStep2"></a>
### Step 2: Adding ions to neutralize the system
Replace **solvent molecules** with **ions** to **neutralize** the system.

In [20]:
# Genion: Adding ions to reach a 0.05 molar concentration
from biobb_gromacs.gromacs.genion import genion

# Create prop dict and inputs/outputs
prop={
    'neutral':True,
    'concentration':0
}
output_genion_gro = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_genion.gro'
output_genion_top_zip = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_genion_top.zip'

# Create and launch bb
genion(input_tpr_path=output_gppion_tpr,
       output_gro_path=output_genion_gro, 
       input_top_zip_path=output_solvate_top_zip,
       output_top_zip_path=output_genion_top_zip, 
       properties=prop)

2025-04-19 22:31:20,521 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.genion Version: 4.2.0
2025-04-19 22:31:20,526 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_complex_gppion.tpr to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/fc3d4b7d-1c25-4456-9a77-b928a8acab05
2025-04-19 22:31:20,528 [MainThread  ] [INFO ]  Copy: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/506c9543-12f6-4ccc-9bf3-34a69ea3df9d.stdin to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/fc3d4b7d-1c25-4456-9a77-b928a8acab05
2025-04-19 22:31:20,538 [MainThread  ] [INFO ]  Extracting: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/prot_aminostep2012_STM_solvate_top.zip
2025-04-19 22:31:20,539 [MainThread  ] [INFO ]  to:
2025-04-19 22:31:20,540 [MainThread  ] [INFO ]

0

### Visualizing 3D structure
Visualizing the **protein-ligand complex** with the newly added **ionic concentration** using **NGL**

In [ ]:
%%script true
#Show protein
view = nglview.show_structure_file(output_genion_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='licorice', radius='.5', selection=ligandCode)
view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<a id="min"></a>
***
## Energetically minimize the system
Energetically minimize the **protein-ligand complex** till reaching a desired potential energy.
- [Step 1](#emStep1): Creating portable binary run file for energy minimization
- [Step 2](#emStep2): Energetically minimize the **protein-ligand complex** till reaching a force of 500 kJ/mol*nm.
- [Step 3](#emStep3): Checking **energy minimization** results. Plotting energy by time during the **minimization** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_gromacs.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="emStep1"></a>
### Step 1: Creating portable binary run file for energy minimization
Method used to run the **energy minimization** is a **steepest descent**, with a **maximum force of 500 kJ/mol\*nm^2**, and a minimization **step size of 1fs**. The **maximum number of steps** to perform if the maximum force is not reached is **5,000 steps**. 

In [ ]:
# Grompp: Creating portable binary run file for mdrun
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
prop = {
    'mdp':{
        'nsteps':'15000',
        'emstep': 0.01,
        'emtol':'500'
    },
    'simulation_type':'minimization'
}
output_gppmin_tpr = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_gppmin.tpr'

# Create and launch bb
grompp(input_gro_path=output_genion_gro,
       input_top_zip_path=output_genion_top_zip,
       output_tpr_path=output_gppmin_tpr,
       properties=prop)

2025-04-19 22:32:01,291 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.grompp Version: 4.2.0
2025-04-19 22:32:01,300 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_genion.gro to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/08530403-6cdb-4127-bc1e-7216794bb830
2025-04-19 22:32:01,320 [MainThread  ] [INFO ]  Extracting: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/prot_aminostep2012_STM_genion_top.zip
2025-04-19 22:32:01,322 [MainThread  ] [INFO ]  to:
2025-04-19 22:32:01,325 [MainThread  ] [INFO ]  ['/home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/08530403-6cdb-4127-bc1e-7216794bb830/STM_posres.itp', '/home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/08530403-6cdb-4127-bc1e-7216794bb830/STMparams.itp', '/home/daneel/gitrepos/gromacs_sim

0

<a id="emStep2"></a>
### Step 2: Running Energy Minimization
Running **energy minimization** using the **tpr file** generated in the previous step.

In [ ]:
# Mdrun: Running minimization
from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_min_trr = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_min.trr'
output_min_gro = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_min.gro'
output_min_edr = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_min.edr'
output_min_log = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_min.log'

# Create and launch bb
mdrun(input_tpr_path=output_gppmin_tpr,
      output_trr_path=output_min_trr, 
      output_gro_path=output_min_gro,
      output_edr_path=output_min_edr, 
      output_log_path=output_min_log,
      use_gpu=True,
      num_threads_omp=nprocs,
      gpu_id='0')

2025-04-19 22:37:00,864 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.mdrun Version: 4.2.0
2025-04-19 22:37:00,874 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_gppmin.tpr to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/08bd6f05-9d37-4129-8385-1e190698a9d6
2025-04-19 22:37:00,877 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -o /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/08bd6f05-9d37-4129-8385-1e190698a9d6/prot_aminostep2012_STM_min.trr -s /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/08bd6f05-9d37-4129-8385-1e190698a9d6/prot_aminostep2012_STM_gppmin.tpr -c /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/08bd6f05-9d37-4129-8385-1e190698a9d6/prot_aminostep2012_STM_min.gro -e /home/daneel/gitrepos/gromacs_sims/Antimicr

0

<a id="emStep3"></a>
### Step 3: Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** by time during the minimization process. 

In [24]:
# GMXEnergy: Getting system energy by time  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_min_ene_xvg = proteinFile.removesuffix('.pdb') +'_'+ligandCode+'_min_ene.xvg'
prop = {
    'terms':  ["Potential"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_min_edr, 
          output_xvg_path=output_min_ene_xvg, 
          properties=prop)

2025-04-19 22:38:17,948 [MainThread  ] [INFO ]  Executing biobb_analysis.gromacs.gmx_energy Version: 4.2.0
2025-04-19 22:38:17,953 [MainThread  ] [INFO ]  Copy: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/prot_aminostep2012_STM_min.edr to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/2641144d-0ffb-4126-94e3-ec8330d5165c
2025-04-19 22:38:17,956 [MainThread  ] [INFO ]  gmx energy -f /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/2641144d-0ffb-4126-94e3-ec8330d5165c/prot_aminostep2012_STM_min.edr -o /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/2641144d-0ffb-4126-94e3-ec8330d5165c/prot_aminostep2012_STM_min_ene.xvg -xvg none < 68a8f6d8-52d1-4d81-b79b-ac7e365cec54/instructions.in

2025-04-19 22:38:18,084 [MainThread  ] [INFO ]  Exit code 0

2025-04-

0

In [25]:
import plotly.graph_objs as go

#Read data from file and filter energy values higher than 1000 kJ/mol
with open(output_min_ene_xvg, 'r') as energy_file:
    x, y = zip(*[
        (float(line.split()[0]), float(line.split()[1]))
        for line in energy_file
        if not line.startswith(("#", "@"))
        if float(line.split()[1]) < 1000
    ])

# Create a scatter plot
fig = go.Figure(data=go.Scatter(x=x, y=y, mode='lines'))

# Update layout
fig.update_layout(title="Energy Minimization",
                  xaxis_title="Energy Minimization Step",
                  yaxis_title="Potential Energy kJ/mol",
                  height=600)

# Show the figure (renderer changes for colab and jupyter)
rend = 'colab' if 'google.colab' in sys.modules else ''
fig.show(renderer=rend)

<a id="nvt"></a>
***
## Equilibrate the system (NVT)
Equilibrate the **protein-ligand complex** system in NVT ensemble (constant Number of particles, Volume and Temperature). To avoid temperature coupling problems, a new *"system"* group will be created including the **protein** + the **ligand** to be assigned to a single thermostatting group.

- [Step 1](#eqNVTStep1): Creating an index file with a new group including the **protein-ligand complex**.
- [Step 2](#eqNVTStep3): Creating portable binary run file for system equilibration
- [Step 3](#eqNVTStep3): Equilibrate the **protein-ligand complex** with NVT ensemble.
- [Step 4](#eqNVTStep4): Checking **NVT Equilibration** results. Plotting **system temperature** by time during the **NVT equilibration** process. 
***
**Building Blocks** used:
- [MakeNdx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.make_ndx) from **biobb_gromacs.gromacs.make_ndx** 
- [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
- [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_gromacs.gromacs.mdrun** 
- [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="eqNVTStep1"></a>
### Step 1: Creating an index file with a new group including the **protein-ligand complex**

In [26]:
# MakeNdx: Creating index file with a new group (protein-ligand complex)
from biobb_gromacs.gromacs.make_ndx import make_ndx

# Create prop dict and inputs/outputs
output_complex_ndx = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_index.ndx'
prop = {
    'selection': "\"Protein\"|\"Other\"" 
}

# Create and launch bb
make_ndx(input_structure_path=output_min_gro,
        output_ndx_path=output_complex_ndx,
        properties=prop)

2025-04-19 22:38:56,071 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.make_ndx Version: 4.2.0
2025-04-19 22:38:56,083 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_min.gro to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/6bd21132-3a73-4b9a-aade-f2dc6a1c15c4
2025-04-19 22:38:56,088 [MainThread  ] [INFO ]  Copy: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/60c055c3-b6d2-4960-8e2d-78d665b4a873.stdin to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/6bd21132-3a73-4b9a-aade-f2dc6a1c15c4
2025-04-19 22:38:56,090 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright make_ndx -f /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/6bd21132-3a73-4b9a-aade-f2dc6a1c15c4/prot_aminostep2012_STM_min.gro -o /home/daneel/gitrepos/gromacs_sims/Antimicrobial

0

<a id="eqNVTStep2"></a>
### Step 2: Creating portable binary run file for system equilibration (NVT)
Note that for the purposes of temperature coupling, the **protein-ligand complex** (*Protein_Other*) is considered as a single entity.

In [ ]:
# Grompp: Creating portable binary run file for NVT System Equilibration
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppnvt_tpr = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'gppnvt.tpr'
prop = {
    'mdp':{
        'nsteps':'50000',
        'tc-grps': 'Protein_Other Water_and_ions',
        'Define': '-DPOSRES -D' + posresifdef
    },
    'simulation_type':'nvt'
}

# Create and launch bb
grompp(input_gro_path=output_min_gro,
       input_top_zip_path=output_genion_top_zip,
       input_ndx_path=output_complex_ndx,
       output_tpr_path=output_gppnvt_tpr,
       properties=prop, restart=True)

2025-04-19 22:50:24,631 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.grompp Version: 4.2.0
2025-04-19 22:50:24,640 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_min.gro to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/19668b3d-d26f-4791-9d10-ad95c1747bec
2025-04-19 22:50:24,649 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_index.ndx to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/19668b3d-d26f-4791-9d10-ad95c1747bec
2025-04-19 22:50:24,667 [MainThread  ] [INFO ]  Extracting: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/prot_aminostep2012_STM_genion_top.zip
2025-04-19 22:50:24,669 [MainThread  ] [INFO ]  to:
2025-04-19 22:50:24,671 [MainThread  ] [INFO ]  ['/home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/19668b3d-d26f-4791

0

<a id="eqNVTStep3"></a>
### Step 3: Running NVT equilibration

In [44]:
# Mdrun: Running NVT System Equilibration 
from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_nvt_trr = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_nvt.trr'
output_nvt_gro = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_nvt.gro'
output_nvt_edr = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_nvt.edr'
output_nvt_log = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_nvt.log'
output_nvt_cpt = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_nvt.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppnvt_tpr,
      output_trr_path=output_nvt_trr,
      output_gro_path=output_nvt_gro,
      output_edr_path=output_nvt_edr,
      output_log_path=output_nvt_log,
      output_cpt_path=output_nvt_cpt,
      use_gpu=True,
      num_threads_omp=nprocs,
      gpu_id='0')

2025-04-19 22:50:29,445 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.mdrun Version: 4.2.0
2025-04-19 22:50:29,455 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STMgppnvt.tpr to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/4d0dbe9e-a9d7-465d-a98d-a508c0d10d0e
2025-04-19 22:50:29,457 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -o /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/4d0dbe9e-a9d7-465d-a98d-a508c0d10d0e/prot_aminostep2012_STM_nvt.trr -s /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/4d0dbe9e-a9d7-465d-a98d-a508c0d10d0e/prot_aminostep2012_STMgppnvt.tpr -c /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/4d0dbe9e-a9d7-465d-a98d-a508c0d10d0e/prot_aminostep2012_STM_nvt.gro -e /home/daneel/gitrepos/gromacs_sims/Antimicrob

0

<a id="eqNVTStep4"></a>
### Step 4: Checking NVT Equilibration results
Checking **NVT Equilibration** results. Plotting **system temperature** by time during the NVT equilibration process. 

In [45]:
# GMXEnergy: Getting system temperature by time during NVT Equilibration  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_nvt_temp_xvg = proteinFile.removesuffix('.pdb') +'_'+ligandCode+'_nvt_temp.xvg'
prop = {
    'terms':  ["Temperature"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_nvt_edr, 
          output_xvg_path=output_nvt_temp_xvg, 
          properties=prop)

2025-04-19 23:15:58,868 [MainThread  ] [INFO ]  Executing biobb_analysis.gromacs.gmx_energy Version: 4.2.0
2025-04-19 23:15:58,873 [MainThread  ] [INFO ]  Copy: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/prot_aminostep2012_STM_nvt.edr to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/a14fd42d-030f-49ad-a1c5-b91f68dc9228
2025-04-19 23:15:58,878 [MainThread  ] [INFO ]  gmx energy -f /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/a14fd42d-030f-49ad-a1c5-b91f68dc9228/prot_aminostep2012_STM_nvt.edr -o /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/a14fd42d-030f-49ad-a1c5-b91f68dc9228/prot_aminostep2012_STM_nvt_temp.xvg -xvg none < 6496a487-bfbd-45e7-b35e-4d41997b2dd9/instructions.in

2025-04-19 23:15:58,968 [MainThread  ] [INFO ]  Exit code 0

2025-04

0

In [46]:
import plotly.graph_objs as go

# Read temperature data from file 
with open(output_nvt_temp_xvg, 'r') as temperature_file:
    x, y = zip(*[
        (float(line.split()[0]), float(line.split()[1]))
        for line in temperature_file
        if not line.startswith(("#", "@"))
    ])

# Create a scatter plot
fig = go.Figure(data=go.Scatter(x=x, y=y, mode='lines+markers'))

fig.update_yaxes(range=[273.15, 310])
# Update layout
fig.update_layout(title="Temperature during NVT Equilibration",
                  xaxis_title="Time (ps)",
                  yaxis_title="Temperature (K)",
                  height=600)

# Show the figure (renderer changes for colab and jupyter)
rend = 'colab' if 'google.colab' in sys.modules else ''
fig.show(renderer=rend)

<a id="npt"></a>
***
## Equilibrate the system (NPT)
Equilibrate the **protein-ligand complex** system in NPT ensemble (constant Number of particles, Pressure and Temperature) .
- [Step 1](#eqNPTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNPTStep2): Equilibrate the **protein-ligand complex** with NPT ensemble.
- [Step 3](#eqNPTStep3): Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_gromacs.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="eqNPTStep1"></a>
### Step 1: Creating portable binary run file for system equilibration (NPT)

In [ ]:
# Grompp: Creating portable binary run file for (NPT) System Equilibration
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppnpt_tpr = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_gppnpt.tpr'
prop = {
    'mdp':{
        'type': 'npt',
        'nsteps':'50000',
        'tc-grps': 'Protein_Other Water_and_ions',
        'Define': '-DPOSRES -D' + posresifdef
    },
    'simulation_type':'npt'
}

# Create and launch bb
grompp(input_gro_path=output_nvt_gro,
       input_top_zip_path=output_genion_top_zip,
       input_ndx_path=output_complex_ndx,
       output_tpr_path=output_gppnpt_tpr,
       input_cpt_path=output_nvt_cpt,
       properties=prop, restart=True)

2025-04-19 23:18:04,946 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.grompp Version: 4.2.0
2025-04-19 23:18:04,953 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_nvt.gro to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/c9f3de79-dd36-479e-96fb-981a14671b29
2025-04-19 23:18:04,956 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_nvt.cpt to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/c9f3de79-dd36-479e-96fb-981a14671b29
2025-04-19 23:18:04,959 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_index.ndx to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/c9f3de79-dd36-479e-96fb-981a14671b29
2025-04-19 23:18:04,967 [MainThread  ] [INFO ]  Extracting: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/prot_aminostep2012_STM_genion_top

0

<a id="eqNPTStep2"></a>
### Step 2: Running NPT equilibration

In [50]:
# Mdrun: Running NPT System Equilibration
from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_npt_trr = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_npt.trr'
output_npt_gro = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_npt.gro'
output_npt_edr = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_npt.edr'
output_npt_log = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_npt.log'
output_npt_cpt = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_npt.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppnpt_tpr,
      output_trr_path=output_npt_trr,
      output_gro_path=output_npt_gro,
      output_edr_path=output_npt_edr,
      output_log_path=output_npt_log,
      output_cpt_path=output_npt_cpt,
      use_gpu=True,
      num_threads_omp=nprocs,
      gpu_id='0')

2025-04-19 23:18:10,789 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.mdrun Version: 4.2.0
2025-04-19 23:18:10,800 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_gppnpt.tpr to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/130b4f07-0959-44a3-9706-1c66b658ab31
2025-04-19 23:18:10,803 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -o /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/130b4f07-0959-44a3-9706-1c66b658ab31/prot_aminostep2012_STM_npt.trr -s /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/130b4f07-0959-44a3-9706-1c66b658ab31/prot_aminostep2012_STM_gppnpt.tpr -c /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/130b4f07-0959-44a3-9706-1c66b658ab31/prot_aminostep2012_STM_npt.gro -e /home/daneel/gitrepos/gromacs_sims/Antimicr

0

<a id="eqNPTStep3"></a>
### Step 3: Checking NPT Equilibration results
Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process. 

In [51]:
# GMXEnergy: Getting system pressure and density by time during NPT Equilibration  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_npt_pd_xvg = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_npt_PD.xvg'
prop = {
    'terms':  ["Pressure","Density"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_npt_edr, 
          output_xvg_path=output_npt_pd_xvg, 
          properties=prop)

2025-04-19 23:23:18,479 [MainThread  ] [INFO ]  Executing biobb_analysis.gromacs.gmx_energy Version: 4.2.0
2025-04-19 23:23:18,483 [MainThread  ] [INFO ]  Copy: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/prot_aminostep2012_STM_npt.edr to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/a3202608-cb5d-4eaa-a22d-550fc3680fbc
2025-04-19 23:23:18,488 [MainThread  ] [INFO ]  gmx energy -f /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/a3202608-cb5d-4eaa-a22d-550fc3680fbc/prot_aminostep2012_STM_npt.edr -o /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/a3202608-cb5d-4eaa-a22d-550fc3680fbc/prot_aminostep2012_STM_npt_PD.xvg -xvg none < fe874454-87b6-43a8-9a41-c17caf107d50/instructions.in

2025-04-19 23:23:18,540 [MainThread  ] [INFO ]  Exit code 0

2025-04-1

0

In [52]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Read pressure and density data from file 
with open(output_npt_pd_xvg,'r') as pd_file:
    x, y, z = zip(*[
        (float(line.split()[0]), float(line.split()[1]), float(line.split()[2]))
        for line in pd_file
        if not line.startswith(("#", "@"))
    ])

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = make_subplots(rows=1, cols=2, print_grid=False)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig.update_layout(
    height=500,
    title='Pressure and Density during NPT Equilibration',
    showlegend=False,
    xaxis1_title='Time (ps)',
    yaxis1_title='Pressure (bar)',
    xaxis2_title='Time (ps)',
    yaxis2_title='Density (Kg*m^-3)'
)

# Show the figure (renderer changes for colab and jupyter)
rend = 'colab' if 'google.colab' in sys.modules else ''
fig.show(renderer=rend)

<a id="free"></a>
***
## Free Molecular Dynamics Simulation
Upon completion of the **two equilibration phases (NVT and NPT)**, the system is now well-equilibrated at the desired temperature and pressure. The **position restraints** can now be released. The last step of the **protein-ligand complex** MD setup is a short, **free MD simulation**, to ensure the robustness of the system. 
- [Step 1](#mdStep1): Creating portable binary run file to run a **free MD simulation**.
- [Step 2](#mdStep2): Run short MD simulation of the **protein-ligand complex**.
- [Step 3](#mdStep3): Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. 
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_gromacs.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_gromacs.gromacs.mdrun** 
 - [GMXRms](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rms) from **biobb_analysis.gromacs.gmx_rms** 
 - [GMXRgyr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rgyr) from **biobb_analysis.gromacs.gmx_rgyr** 
***

<a id="mdStep1"></a>
### Step 1: Creating portable binary run file to run a free MD simulation

In [53]:
# Grompp: Creating portable binary run file for mdrun
from biobb_gromacs.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
prop = {
    'mdp':{
        #'nsteps':'500000' # 1 ns (500,000 steps x 2fs per step)
        #'nsteps':'5000' # 10 ps (5,000 steps x 2fs per step)
        'nsteps':'5000000' # 10 ns (5000000 steps x 2fs per step)
    },
    'simulation_type':'free'
}
output_gppmd_tpr = proteinFile.removesuffix('.pdb')+'_'+ligandCode + '_gppmd.tpr'

# Create and launch bb
grompp(input_gro_path=output_npt_gro,
       input_top_zip_path=output_genion_top_zip,
       output_tpr_path=output_gppmd_tpr,
       input_cpt_path=output_npt_cpt,
       properties=prop)

2025-04-19 23:28:10,154 [MainThread  ] [INFO ]  Executing biobb_gromacs.gromacs.grompp Version: 4.2.0
2025-04-19 23:28:10,157 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_npt.gro to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/8a71c7d1-c22c-45fa-8dc3-885de75113cd
2025-04-19 23:28:10,159 [MainThread  ] [INFO ]  Copy: prot_aminostep2012_STM_npt.cpt to /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/8a71c7d1-c22c-45fa-8dc3-885de75113cd
2025-04-19 23:28:10,165 [MainThread  ] [INFO ]  Extracting: /home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/prot_aminostep2012_STM_genion_top.zip
2025-04-19 23:28:10,166 [MainThread  ] [INFO ]  to:
2025-04-19 23:28:10,167 [MainThread  ] [INFO ]  ['/home/daneel/gitrepos/gromacs_sims/Antimicrobial_Peptides/Pseudomonas_antibiotics/simulation/aminostep2012/8a71c7d1-c22c-45fa-8

0

<a id="mdStep2"></a>
### Step 2: Running short free MD simulation

In [ ]:
# Mdrun: Running free dynamics
from biobb_gromacs.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_md_trr = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_md.trr'
output_md_gro = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_md.gro'
output_md_edr = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_md.edr'
output_md_log = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_md.log'
output_md_cpt = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_md.cpt'
output_md_xtc = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_md.xtc'

# Create and launch bb
mdrun(input_tpr_path=output_gppmd_tpr,
      output_trr_path=output_md_trr,
      output_xtc_path=output_md_xtc,
      output_gro_path=output_md_gro,
      output_edr_path=output_md_edr,
      output_log_path=output_md_log,
      output_cpt_path=output_md_cpt)

<a id="mdStep3"></a>
### Step 3: Checking free MD simulation results
Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. **RMSd** against the **experimental structure** (input structure of the pipeline) and against the **minimized and equilibrated structure** (output structure of the NPT equilibration step).

In [ ]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against minimized and equilibrated snapshot (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_first = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_rms_first.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rms(input_structure_path=output_gppmd_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rms_first, 
          properties=prop)

In [ ]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against experimental structure (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_exp = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_rms_exp.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rms(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rms_exp, 
          properties=prop)

In [ ]:
import plotly.graph_objs as go

# Read RMS vs first snapshot data from file 
with open(output_rms_first,'r') as rms_first_file:
    x, y = zip(*[
        (float(line.split()[0]), float(line.split()[1]))
        for line in rms_first_file
        if not line.startswith(("#", "@"))
    ])

# Read RMS vs experimental structure data from file 
with open(output_rms_exp,'r') as rms_exp_file:
    x2, y2 = zip(*[
        (float(line.split()[0]), float(line.split()[1]))
        for line in rms_exp_file
        if not line.startswith(("#", "@"))
    ])

fig = make_subplots()
fig.add_trace(go.Scatter(x=x, y=y, mode="lines+markers", name="RMSd vs first"))
fig.add_trace(go.Scatter(x=x, y=y2, mode="lines+markers", name="RMSd vs exp"))

# Set layout including height
fig.update_layout(
    title="RMSd during free MD Simulation",
    xaxis=dict(title="Time (ps)"),
    yaxis=dict(title="RMSd (nm)"),
    height=600
)

# Show the figure (renderer changes for colab and jupyter)
rend = 'colab' if 'google.colab' in sys.modules else ''
fig.show(renderer=rend)

In [ ]:
# GMXRgyr: Computing Radius of Gyration to measure the protein compactness during the free MD simulation 

from biobb_analysis.gromacs.gmx_rgyr import gmx_rgyr

# Create prop dict and inputs/outputs
output_rgyr = proteinFile.removesuffix('.pdb')+'_'+ligandCode+'_rgyr.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rgyr(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rgyr, 
          properties=prop)

In [ ]:
import plotly.graph_objs as go

# Read Rgyr data from file
with open(output_rgyr, 'r') as rgyr_file:
    x, y = zip(*[
        (float(line.split()[0]), float(line.split()[1]))
        for line in rgyr_file
        if not line.startswith(("#", "@"))
    ])

# Create a scatter plot
fig = go.Figure(data=go.Scatter(x=x, y=y, mode='lines+markers'))

# Update layout
fig.update_layout(title="Radius of Gyration",
                  xaxis_title="Time (ps)",
                  yaxis_title="Rgyr (nm)",
                  height=600)

# Show the figure (renderer changes for colab and jupyter)
rend = 'colab' if 'google.colab' in sys.modules else ''
fig.show(renderer=rend)

<a id="post"></a>
***
## Post-processing and Visualizing resulting 3D trajectory
Post-processing and Visualizing the **protein-ligand complex system** MD setup **resulting trajectory** using **NGL**
- [Step 1](#ppStep1): *Imaging* the resulting trajectory, **stripping out water molecules and ions** and **correcting periodicity issues**.
- [Step 2](#ppStep2): Generating a *dry* structure, **removing water molecules and ions** from the final snapshot of the MD setup pipeline.
- [Step 3](#ppStep3): Visualizing the *imaged* trajectory using the *dry* structure as a **topology**. 
***
**Building Blocks** used:
 - [GMXImage](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_image) from **biobb_analysis.gromacs.gmx_image** 
 - [GMXTrjConvStr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_trjconv_str) from **biobb_analysis.gromacs.gmx_trjconv_str** 
***

<a id="ppStep1"></a>
### Step 1: *Imaging* the resulting trajectory.
Stripping out **water molecules and ions** and **correcting periodicity issues**  

In [ ]:
# GMXImage: "Imaging" the resulting trajectory
#           Removing water molecules and ions from the resulting structure
from biobb_analysis.gromacs.gmx_image import gmx_image

# Create prop dict and inputs/outputs
output_imaged_traj = proteinFile.removesuffix('.pdb')+'_imaged_traj.xtc'
prop = {
    'center_selection':  'Protein_Other',
    'output_selection': 'Protein_Other',
    'pbc' : 'mol',
    'center' : True
}

# Create and launch bb
gmx_image(input_traj_path=output_md_xtc,
         input_top_path=output_gppmd_tpr,
         input_index_path=output_complex_ndx,
         output_traj_path=output_imaged_traj, 
          properties=prop)

<a id="ppStep2"></a>
### Step 2: Generating the output *dry* structure.
**Removing water molecules and ions** from the resulting structure

In [ ]:
# GMXTrjConvStr: Converting and/or manipulating a structure
#                Removing water molecules and ions from the resulting structure
#                The "dry" structure will be used as a topology to visualize 
#                the "imaged dry" trajectory generated in the previous step.
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str

# Create prop dict and inputs/outputs
output_dry_gro = proteinFile.removesuffix('.pdb')+'_md_dry.gro'
prop = {
    'selection':  'Protein_Other'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_md_gro,
                input_top_path=output_gppmd_tpr,
                input_index_path=output_complex_ndx,
                output_str_path=output_dry_gro, 
                properties=prop)

<a id="ppStep3"></a>
### Step 3: Visualizing the generated dehydrated trajectory.
Using the **imaged trajectory** (output of the [Post-processing step 1](#ppStep1)) with the **dry structure** (output of the [Post-processing step 2](#ppStep2)) as a topology.

In [ ]:
%%script true
# Show trajectory
view = nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_imaged_traj, output_dry_gro), gui=True)
view

<a id="output"></a>
## Output files

Important **Output files** generated:
 - **output_md_gro** : **Final structure** (snapshot) of the MD setup protocol.
 - **output_md_trr** : **Final trajectory** of the MD setup protocol.
 - **output_md_xtc** : **Final trajectory** of the MD setup protocol (single precision positions only).
 - **output_md_cpt** : **Final checkpoint file**, with information about the state of the simulation. It can be used to **restart** or **continue** a MD simulation.
 - **output_gppmd_tpr** : **Final tpr file**, GROMACS portable binary run input file. This file contains the starting structure of the **MD setup free MD simulation step**, together with the molecular topology and all the simulation parameters. It can be used to **extend** the simulation.
 - **output_genion_top_zip** : **Final topology** of the MD system. It is a compressed zip file including a **topology file** (.top) and a set of auxiliary **include topology** files (.itp).

**Analysis** (MD setup check) output files generated:
 - **output_rms_first** : **Root Mean Square deviation (RMSd)** against **minimized and equilibrated structure** of the final **free MD run step**.
 - **output_rms_exp** : **Root Mean Square deviation (RMSd)** against **experimental structure** of the final **free MD run step**.
 - **output_rgyr** : **Radius of Gyration** of the final **free MD run step** of the **setup pipeline**.
 

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)


## Post-Processing

Optional. Save all declared variables to pickle.

In [ ]:
import pickle
all_variables = globals()
variables_to_pickle = {k: v for k, v in all_variables.items() if not k.startswith('_') and not isinstance(v, type(pickle))}
with open('notebook_variables.pkl', 'wb') as f:
        pickle.dump(variables_to_pickle, f)